In [1]:
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git

Cloning into 'PyTorch-Architectures'...
remote: Enumerating objects: 405, done.
remote: Counting objects: 100% (405/405), done.
remote: Compressing objects: 100% (242/242), done.
remote: Total 405 (delta 256), reused 283 (delta 149), pack-reused 0
Receiving objects: 100% (405/405), 89.05 KiB | 747.00 KiB/s, done.
Resolving deltas: 100% (256/256), done.


In [1]:
%cd PyTorch-Architectures/modeling_openai/

/content/PyTorch-Architectures/modeling_openai


In [ ]:
! pip install transformers
! pip install ftfy
! pip install spacy
! pip install datasets

In [2]:
import time
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import OpenAIGPTTokenizer
from model import OpenAIGPTLMHeadModel
from config_openai import OpenAIGPTConfig
config = OpenAIGPTConfig()
tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")
# pad_token is not set by default
tokenizer.pad_token = '[PAD]'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
from datasets import load_dataset
dataset = load_dataset('cnn_dailymail', '3.0.0')

Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0128610a44e10f25b4af6689441c72af86205282d26399642f7db38fa7535602)


In [4]:
##############################################
# PyTorch Dataset
class CustomDataset(Dataset):

    def __init__(self, texts, tokenizer):
        self.texts = texts
        self.tokenizer = tokenizer
        self.list_texts = []
        self.build()

    def __len__(self):
        return len(self.list_texts)

    def __getitem__(self, index):
        ids = self.list_texts[index]['input_ids']
        mask = self.list_texts[index]['attention_mask']
        
        return{
                'ids': torch.tensor(ids, dtype=torch.long),
                'mask': torch.tensor(mask, dtype=torch.long)
                }

    def build(self):
        for text in self.texts:
            self.list_texts.append(self.tokenizer(text, max_length=64, pad_to_max_length=True, truncation=True))
################################################

In [5]:
texts_train = dataset['train']['article'][:10000]
texts_valid = dataset['validation']['article'][:1000]

start_time = time.time()
train_dataset = CustomDataset(texts_train, tokenizer)
valid_dataset = CustomDataset(texts_valid, tokenizer)
print("Time Taken = ", (time.time() - start_time) / 60)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Time Taken =  2.3026591738065085


In [7]:
train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=8)
valid_loader = DataLoader(dataset=valid_dataset, shuffle=False, batch_size=8)
print("Training Samples = ", len(train_loader))
print("Validation Samples = ", len(valid_loader))

Training Samples =  1250
Validation Samples =  125


In [8]:
model = OpenAIGPTLMHeadModel(config).to(device)
total_params = sum(p.numel() for p in model.parameters())
print("Total Parameters = ", total_params)

Total Parameters =  74007552


In [9]:
EPOCHS = 5
LEARNING_RATE = 2.5e-4
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [10]:
def compute_loss(model, data_loader, device):
  total_loss = 0
  model.eval()
  for sample in data_loader:
    ids = sample['ids'].to(device)
    mask = sample['mask'].to(device)

    outputs = model(input_ids=ids, attention_mask=mask, labels=ids)
    loss = outputs[0]
    total_loss += loss.item()
  average_loss = total_loss / len(data_loader)
  return average_loss

start_time = time.time()
for epoch in range(EPOCHS):
  model.train()
  for idx, sample in enumerate(train_loader):
    ids = sample['ids'].to(device)
    mask = sample['mask'].to(device)

    optimizer.zero_grad()
    outputs = model(input_ids=ids, attention_mask=mask, labels=ids)
    loss = outputs[0]

    # LOGGING
    if idx % 500 == 0:
      print("Batch: %04d/%04d || Epoch: %03d/%03d || Loss: %.3f" % (idx, len(train_loader), epoch, EPOCHS, loss.item()))
    
    loss.backward()
    optimizer.step()

  model.eval()
  with torch.set_grad_enabled(False):
    valid_loss = compute_loss(model, valid_loader, device)
    print("Validation Average Loss: ", valid_loss)
  
  epoch_elapsed_time = (time.time() - start_time) / 60
  print("Epoch Elapsed Time: ", epoch_elapsed_time)
total_elapsed_time = (time.time() - start_time) / 60
print("Total Training Elapsed Time: ", total_elapsed_time)

Batch: 000/1250 || Epoch: 000/005 || Loss: 10.763
Batch: 500/1250 || Epoch: 000/005 || Loss: 5.851
Batch: 1000/1250 || Epoch: 000/005 || Loss: 6.006
Validation Average Loss:  5.818221267700196
Epoch Elapsed Time:  4.116805644830068
Batch: 000/1250 || Epoch: 001/005 || Loss: 5.253
Batch: 500/1250 || Epoch: 001/005 || Loss: 5.417
Batch: 1000/1250 || Epoch: 001/005 || Loss: 5.270
Validation Average Loss:  5.475076210021973
Epoch Elapsed Time:  8.238336928685506
Batch: 000/1250 || Epoch: 002/005 || Loss: 4.346
Batch: 500/1250 || Epoch: 002/005 || Loss: 4.931
Batch: 1000/1250 || Epoch: 002/005 || Loss: 4.698
Validation Average Loss:  5.348075637817383
Epoch Elapsed Time:  12.362340748310089
Batch: 000/1250 || Epoch: 003/005 || Loss: 3.807
Batch: 500/1250 || Epoch: 003/005 || Loss: 4.117
Batch: 1000/1250 || Epoch: 003/005 || Loss: 4.159
Validation Average Loss:  5.350865081787109
Epoch Elapsed Time:  16.48643560409546
Batch: 000/1250 || Epoch: 004/005 || Loss: 3.408
Batch: 500/1250 || Epoch:

In [51]:
# Testing the model TODO (Needs more training as of now)
import torch.nn.functional as F
text = ["his"]
dataset = CustomDataset(text, tokenizer)
data_loader = DataLoader(dataset=dataset, shuffle=False, batch_size=1)
model.eval()
with torch.set_grad_enabled(False):
  for sample in data_loader:
    ids = sample['ids'].to(device)
    mask = sample['mask'].to(device)
    outputs = model(input_ids=ids, attention_mask=mask)
    logits = outputs[0]
    logits = logits.view(-1, logits.size(-1))
    # print(logits.shape)
    probas = F.softmax(logits, dim=1)
    _, predicted_logit = torch.max(logits, 1)
    predicted_logit = predicted_logit[0]
    print(tokenizer.decode([predicted_logit]))

wife


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
